# Federated Linear Model with Scikit-learn on HIGGS Dataset

## Introduction to Scikit-learn, tabular data, and federated k-Means
### Scikit-learn
This example shows how to use [NVIDIA FLARE](https://nvflare.readthedocs.io/en/main/index.html) on tabular data.
It uses [Scikit-learn](https://scikit-learn.org/),
a widely used open-source machine learning library that supports supervised 
and unsupervised learning.
### Tabular data
The data used in this example is tabular in a format that can be handled by [pandas](https://pandas.pydata.org/), such that:
- rows correspond to data samples
- the first column represents the label 
- the other columns cover the features.    

Each client is expected to have one local data file containing both training 
and validation samples. To load the data for each client, the following 
parameters are expected by the local learner:
- data_file_path: string, the full path to the client's data file 
- train_start: int, start row index for the training set
- train_end: int, end row index for the training set
- valid_start: int, start row index for the validation set
- valid_end: int, end row index for the validation set

### Federated Linear Model
This example shows the use of [linear classifiers with SGD training](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) in a federated scenario.
Under this setting, federated learning can be formulated as a [FedAvg](https://arxiv.org/abs/1602.05629) process with local training that each client optimizes the local model starting from global parameters with SGD. 
This can be achieved by setting the `warm_start` flag of SGDClassifier to 
`True` in order to allow repeated fitting of the classifiers to the local data.

Below we listed steps to run this example.

## 1. Setup NVFLARE

Follow the [Getting_Started](https://nvflare.readthedocs.io/en/main/getting_started.html) to setup virtual environment and install NVFLARE

We also provide a [Notebook](../../nvflare_setup.ipynb) for this setup process. 

Assume you have already setup the venv, lets first install required packages.

In [ ]:
%pip install -r requirements.txt

## 2. Data preparation 
The examples illustrate a binary classification task based on [HIGGS dataset](https://mlphysics.ics.uci.edu/data/higgs/).
This dataset contains 11 million instances, each with 28 attributes.
By default, we assume the dataset is downloaded, uncompressed, and stored in `DATASET_ROOT/HIGGS.csv`.
Please note that the UCI's website may experience occasional downtime.


## 3. Prepare clients' configs with proper data split information 
We are using NVFlare's FL simulator to run the following experiments. Here we simulate 5 clients with uniform data split. Since HIGGS dataset is already randomly recorded, the split is done sequentially. Please change the DATASET_ROOT to the correct local path

In [ ]:
%env DATASET_ROOT=/data
!python3 ./utils/prepare_job_config.py \
        --task_name "sklearn_linear" \
        --data_path "${DATASET_ROOT}/HIGGS.csv" \
        --site_num 5 \
        --valid_frac 0.1 \
        --split_method "uniform"

## 4. Run simulated experiment
Now that we have the job configs ready, we run the experiment using Simulator.

Simulator can either be used with CLI command: 

In [ ]:
! nvflare simulator ./jobs/sklearn_linear_5_uniform -w /tmp/nvflare/sklearn_linear_higgs -n 5 -t 5

or via Simulator API:

In [ ]:
import os
from nvflare.private.fed.app.simulator.simulator_runner import SimulatorRunner  

simulator = SimulatorRunner(
    job_folder=f"./jobs/sklearn_linear_5_uniform",
    workspace="/tmp/nvflare/sklearn_linear_higgs",
    n_clients=5,
    threads=5
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

## 5. Result visualization
Model accuracy is computed as the homogeneity score between the cluster formed and the ground truth label, which can be visualized in tensorboard.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/nvflare/sklearn_linear_higgs